In [3]:
import pandas as pd

# cargar DataSet con los tweets de entrenamiento
train_mx = pd.read_csv('mx-train-data-non-contextual.csv', names=['texto'],  header=0)# , nrows=30, header=1) 
# cargar las clases de cada tweet de entrenamiento
sol = pd.read_csv('mx-train-outputs.sol', names=["clase"])

train_mx.shape, sol.shape, train_mx.head(5), sol.head(5)

((5060, 1),
 (5060, 1),
                                                texto
 0  Soy el Clint Eastwood de los Puentes de Madiso...
 1  Actualmente ya pasó de moda la pucha joto, aho...
 2  ¿Es cierto esto? Y no me refiero a lo que dijo...
 3  Vuela pega y esquiva... la neta está de la ver...
 4  Mejor puto disfraz de la noche!!!! 👊👊👊Por terc...,
    clase
 0      0
 1      1
 2      0
 3      0
 4      0)

In [4]:
import sklearn.model_selection as model_selection

# split del datataset completo en entrenamiento | test
X_train, X_test, y_train, y_test = model_selection.train_test_split(train_mx, sol, train_size=0.8, random_state=True)

# Juntar texto|clase y re-ordenar aleatoriamente 
X_y_train_shuffled = X_train.join(y_train).sample(frac=1)

X_train.shape, X_y_train_shuffled.shape, X_y_train_shuffled.head(5), X_y_train_shuffled.loc[[1]]

((4048, 1),
 (4048, 2),
                                                   texto  clase
 311   En estos momentos de la vida es dónde me pregu...      0
 2863  @USUARIO Que andas de payasa jaja queriendote ...      0
 2300  Los subtítulos en español de The Fosters están...      0
 2346  Pague infracciones del bmw... y lo lleve a ver...      0
 396   Vergaa neta me caga que todos los pinches años...      0,
                                                texto  clase
 1  Actualmente ya pasó de moda la pucha joto, aho...      1)

In [5]:
with pd.option_context('display.max_colwidth', -1):
    print(X_y_train_shuffled.loc[[254]]['texto'])
    print(sol.loc[[254]])

254     @USUARIO @USUARIO @USUARIO Después de todo hay ratas de alcantarilla de 4 patas menos feas que estas RATAS de dos patas. ASQUEROSOS
Name: texto, dtype: object
     clase
254  1    


In [8]:
# aumentado de X_train / y_train con bi-frases
_ = X_y_train_shuffled.iloc[0]
train_bi_frase = pd.DataFrame([_], columns = ['texto', 'clase'])
train_bi_frase

for index, row in X_y_train_shuffled.iloc[1:].iterrows():
    try:
        train_bi_frase =  train_bi_frase.append(row)
        if _['clase'] == row['clase']:
            bi_frase = {'texto': _['texto'] + row['texto'], 'clase':row['clase']}
            train_bi_frase = train_bi_frase.append(bi_frase, ignore_index=True)
            _ =  {'texto': '_VACIA_', 'clase': None}
        else:   
            _ = row
    except Exception as exc:
        print(exc)
        _ = row
        continue
        
train_bi_frase.shape

(5504, 2)

In [10]:
# solo para checar algunas bi-frases
print(train_bi_frase.loc[(train_bi_frase['clase'] == 0)].head(5))
print(train_bi_frase.loc[(train_bi_frase['clase'] == 1)].head(5))

                                               texto  clase
0  En estos momentos de la vida es dónde me pregu...      0
1  @USUARIO Que andas de payasa jaja queriendote ...      0
2  En estos momentos de la vida es dónde me pregu...      0
3  Los subtítulos en español de The Fosters están...      0
4  Pague infracciones del bmw... y lo lleve a ver...      0
                                                texto  clase
7   pues si soy joto, me vale verga Jajajaja creo ...      1
9   Si te vas a ir a la verga de mi lado déjame ig...      1
13  chinguen su madre putos chilenos de mierda, #M...      1
14  Cuando vas a ver una pelicula todos esperan si...      1
15  chinguen su madre putos chilenos de mierda, #M...      1


In [11]:
# ver una bifrase completa por índice
with pd.option_context('display.max_colwidth', -1):
    print(train_bi_frase.loc[[12]]['texto'])
    print(train_bi_frase.loc[[19]]['texto'])

12    El fruto del servicio es la paz, Madre Teresa de Calcuta.Primera vez que me dan asueto el día de muertos y yo de viaje de trabajo no mamar
Name: texto, dtype: object
19    Dicen que el que madura se arruga... Pues yo ya me ando poniendo bien madurito de los párpados.
Name: texto, dtype: object


In [12]:
# re-split del train_bi_frase

X_train = train_bi_frase['texto']
y_train = train_bi_frase['clase']

X_train = X_train.squeeze() 
X_test = X_test.squeeze() 
y_train = y_train.squeeze() 
y_test = y_test.squeeze()

X_train.shape, y_train.shape, X_train.head(), y_train.head()

((5504,),
 (5504,),
 0    En estos momentos de la vida es dónde me pregu...
 1    @USUARIO Que andas de payasa jaja queriendote ...
 2    En estos momentos de la vida es dónde me pregu...
 3    Los subtítulos en español de The Fosters están...
 4    Pague infracciones del bmw... y lo lleve a ver...
 Name: texto, dtype: object,
 0    0
 1    0
 2    0
 3    0
 4    0
 Name: clase, dtype: int64)

In [13]:
# modelos para EvomSA

from EvoMSA.utils import download
from EvoMSA.base import EvoMSA

from EvoMSA.model import TextModelInv

from sklearn import metrics
from sklearn.metrics import classification_report

haha = download('haha2018_Es.evomsa')
mexa3t = download('mexa3t2018_aggress_Es.evomsa')
misoginia = download('misoginia_Es.evomsa')

In [15]:
# Clasificador con Modelos
# "text_model_inv.TextModelInv"
# [TextModelInv,"sklearn.svm.LinearSVC"],
evo = EvoMSA(TR=True, B4MSA=False, lang='es', Aggress=True,
                 stacked_method='sklearn.naive_bayes.GaussianNB',
                 models=[
                     [TextModelInv,"sklearn.svm.LinearSVC"],
                     [misoginia, "sklearn.svm.LinearSVC"],
                     [haha, "sklearn.svm.LinearSVC"] ,
                     [mexa3t, "sklearn.svm.LinearSVC"]
                 ])

import time
start = time.time()
# Entrenamiento X_train | y_train
evo.fit(X_train, y_train)    
print(time.time() - start, "seconds")

# Predicción con X_test
pred = evo.predict(X_test)
print(pred)

C:\Users\Calderas\Anaconda3\lib\site-packages\sklearn\base.py:315: UserWarning: Trying to unpickle estimator LinearSVC from version 0.22.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\Calderas\Anaconda3\lib\site-packages\sklearn\base.py:315: UserWarning: Trying to unpickle estimator GaussianNB from version 0.22.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
100%|██████████| 3/3 [00:33<00:00, 11.24s/it]

100%|██████████| 1/1 [00:17<00:00, 17.42s/it]

100%|██████████| 4/4 [00:36<00:00,  9.23s/it]

  0%|          | 0/6 [00:00<?, ?it/s]

172.6022012233734 seconds


100%|██████████| 3/3 [00:09<00:00,  3.23s/it]

100%|██████████| 1/1 [00:05<00:00,  5.17s/it]

100%|██████████| 4/4 [00:09<00:00,  2.49s/it]

100%|██████████| 6/6 [00:00<00:00, 375.98it/s]

[0 0 1 ... 0 0 1]


In [16]:
## 
print("       f1 score macro:", metrics.f1_score(y_test, pred, average="macro"))
print("precision score macro:", metrics.precision_score(y_test, pred, average="macro"))
print("   recall score macro:", metrics.recall_score(y_test, pred, average="macro"))  
print("       f1 score micro:", metrics.f1_score(y_test, pred, average="micro"))
print("precision score micro:", metrics.precision_score(y_test, pred, average="micro"))
print("   recall score micro:", metrics.recall_score(y_test, pred, average="micro"))  

       f1 score macro: 0.7471953201985903
precision score macro: 0.7415315703840197
   recall score macro: 0.7540005669080321
       f1 score micro: 0.8033596837944664
precision score micro: 0.8033596837944664
   recall score micro: 0.8033596837944664
